In [1]:
import pandas as pd
import numpy as np
import math
from numba import njit

# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs,ys,degs,tx,ty):
    n=xs.size; V=tx.size
    mnx=1e300; mny=1e300; mxx=-1e300; mxy=-1e300
    for i in range(n):
        r=degs[i]*math.pi/180.0
        c=math.cos(r); s=math.sin(r)
        xi=xs[i]; yi=ys[i]
        for j in range(V):
            X=c*tx[j]-s*ty[j]+xi
            Y=s*tx[j]+c*ty[j]+yi
            if X<mnx: mnx=X
            if X>mxx: mxx=X
            if Y<mny: mny=Y
            if Y>mxy: mxy=Y
    side=max(mxx-mnx,mxy-mny)
    return side*side/n

def strip(a):
    return np.array([float(str(v).replace("s","")) for v in a],np.float64)

def score_submission(filepath):
    df = pd.read_csv(filepath)
    df["N"] = df["id"].astype(str).str.split("_").str[0].astype(int)
    tx, ty = make_polygon_template()
    total = 0
    for n, g in df.groupby("N"):
        xs = strip(g["x"].to_numpy())
        ys = strip(g["y"].to_numpy())
        ds = strip(g["deg"].to_numpy())
        sc = score_group(xs, ys, ds, tx, ty)
        total += sc
    return total

# Score the pre-optimized submissions
print("Scoring pre-optimized submissions...")
for f in ['/home/code/preoptimized/submission.csv', '/home/code/preoptimized/smartmanoj_submission.csv']:
    try:
        score = score_submission(f)
        print(f"{f}: {score:.6f}")
    except Exception as e:
        print(f"{f}: Error - {e}")

Scoring pre-optimized submissions...


/home/code/preoptimized/submission.csv: 70.647327
/home/code/preoptimized/smartmanoj_submission.csv: 70.743774
